In [1]:
from random import seed, randint
import ast; import sys; from time import process_time
import numpy as np; import verbose_module
from pickle import load, dump
from copy import deepcopy

sys.path.append('../../.')
import pIRPgym

########################     Instance generator and Environment     #########################
### pIRP model's parameters
# Stochasticity
stochastic_params = ['d','q']; look_ahead = ['d','q']

# Historical data
historical_data = ['*']

# Other parameters
backorders = 'backorders'

# Random Instance
q_params = {'dist': 'c_uniform', 'r_f_params': [16,30]}         # Supply
p_params = {'dist': 'd_uniform', 'r_f_params': [20,61]}         # Purchase costs at suppliers'

d_params = {'dist': 'log-normal', 'r_f_params': [3,1]}          # Demand

h_params = {'dist': 'd_uniform', 'r_f_params': [20,40]}        # Holding costs

### Environment 
# Creating environment object
inventory = True; routing = True; perishability = "ages"
env = pIRPgym.steroid_IRP(routing, inventory, perishability)

# Environment check

In [2]:
verbose = False; agg = False; hold_cost = True; save = True

In [3]:
weights = [[1,0,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1]] + [[0.5,0.1,0.1,0.2,0.1]] + [[round(i*0.1,4)]+[round((1-i*0.1)/4,4) for j in range(4)] for i in range(11)]

objectives = ["costs","climate","water","land","fossil"]

for w in weights:
    rewards = dict(); actions = dict(); la_decisions = dict(); states = dict(); backo = dict(); perished = dict(); norm_matrix = dict()
    env_config = {'M':7,'K':7,'T':10,'F':8,'Q':1500,'S':5,'LA_horizon':4,'d_max':2000,'hist_window':60, 'theta':0.9}

    stoch_rd_seed = 40526361; det_rd_seed = 10 # Random seeds
    obj_weights = dict(zip(objectives,w))

    # Creating instance generator object
    inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params, historical_data, backorders, env_config = env_config, sustainability=True)
    inst_gen.generate_random_instance(det_rd_seed, stoch_rd_seed, q_params=q_params, p_params=p_params, d_params=d_params, h_params=h_params)

    # Resetting the environment
    states[1] = env.reset(inst_gen, return_state=True)

    done = False; day = 1
    while not done:

        ''' Objectives Pay-off Matrix and Normalization '''
        pIRPgym.Compromise_Programming.normalize_objectives(inst_gen, env, hold_cost=hold_cost)
        norm_matrix[day] = deepcopy(env.norm_matrix)

        ''' Purchase '''
        [purchase,demand_compliance], la_decisions[day] = pIRPgym.Inventory.Stochastic_Rolling_Horizon(states[day], env, inst_gen, objs=obj_weights, hold_cost=hold_cost)

        ''' Routing '''
        GA_routes,GA_obj,GA_info,GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase, inst_gen, env.t, return_top=False, rd_seed=0, time_limit=30)

        ''' Complete Action '''        
        action = {'routing':GA_routes,'purchase':purchase,'demand_compliance':demand_compliance}

        if verbose:
            if day == 1: verbose_module.headers.print_simple_header(w)
            print(f"\n------- Day {day}")
            verbose_module.objectives_performance.show_normalization(inst_gen,env)
            verbose_module.objectives_performance.show_balanced_solution(inst_gen,env,la_decisions[day],hold_cost=hold_cost)
        
        states[day+1], rewards[day], done, actions[day], (backo[day], perished[day])  = env.step(action,inst_gen,aggregated = agg)
        
        #print(f"Done day {day}")
        day += 1

    if save:
        other_path = True
        verbose_module.export_results.export_instance_parameters(w, 1, inst_gen, other_path)
        verbose_module.export_results.export_actions(w, 1, actions, other_path)
        verbose_module.export_results.export_lookahead_decisions(w, 1, la_decisions, other_path)
        verbose_module.export_results.export_inventory(w, 1, states, other_path)
        verbose_module.export_results.export_rewards(w, 1, rewards, other_path)
        verbose_module.export_results.export_backorders(w, 1, backo, other_path)
        verbose_module.export_results.export_perished(w, 1, perished, other_path)
        verbose_module.export_results.export_norm_matrix(w, 1, norm_matrix, other_path)
    
    print(f"Done weights {w}")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-07
Done weights [1, 0, 0, 0, 0]
Done weights [0, 1, 0, 0, 0]
Done weights [0, 0, 1, 0, 0]
Done weights [0, 0, 0, 1, 0]
Done weights [0, 0, 0, 0, 1]
Done weights [0.5, 0.1, 0.1, 0.2, 0.1]
Done weights [0.0, 0.25, 0.25, 0.25, 0.25]
Done weights [0.1, 0.225, 0.225, 0.225, 0.225]
Done weights [0.2, 0.2, 0.2, 0.2, 0.2]
Done weights [0.3, 0.175, 0.175, 0.175, 0.175]
Done weights [0.4, 0.15, 0.15, 0.15, 0.15]
Done weights [0.5, 0.125, 0.125, 0.125, 0.125]
Done weights [0.6, 0.1, 0.1, 0.1, 0.1]
Done weights [0.7, 0.075, 0.075, 0.075, 0.075]
Done weights [0.8, 0.05, 0.05, 0.05, 0.05]
Done weights [0.9, 0.025, 0.025, 0.025, 0.025]
Done weights [1.0, 0.0, 0.0, 0.0, 0.0]


# Scenarios parameter check

In [ ]:
env_config = {'M':5,'K':5,'T':5,'F':8,'Q':1500,'S':5,'LA_horizon':4,'d_max':2000,'hist_window':10}
st_seeds = [10,20,30,40,50]; dt_seed = 80

generators = list()
for s in st_seeds:
    generators.append(pIRPgym.instance_generator(look_ahead, stochastic_params, historical_data, backorders, env_config = env_config, sustainability=True))
    generators[-1].generate_random_instance(dt_seed, s, q_params=q_params, p_params=p_params, d_params=d_params, h_params=h_params)

S = list(range(1,len(st_seeds)+1))

### Deterministic Parameters

In [ ]:
verbose_module.headers.print_header("Availabilities (M_kt)")
print("\t",*[" "*3+f"s{s}"+" "*4 for s in S],sep="\t")
for k in generators[0].Products:
    print(f"\tk{k}:",*["-".join([str(i) for i in generators[s-1].M_kt[k,0]])+" "*(9-len(generators[s-1].M_kt[k,0])) for s in S],sep="\t")

verbose_module.headers.print_header("Historical Data (hist_d, hist_q, hist_p)")
print("\tDemand")
for s in S:
    print(f"\t\ts{s}:",{f"k{k}":generators[s-1].hist_d[0][k] for k in generators[0].Products},sep="\t")
print("\n\tAvailable Quantities")
for s in S:
    print(f"\t\ts{s}:",{f"(i{i},k{k})":generators[s-1].hist_q[0][i,k] for k in generators[0].Products for i in generators[0].Suppliers},sep="\t")
print("\n\tPurchase Prices")
for s in S:
    print(f"\t\ts{s}:",{f"(i{i},k{k})":generators[s-1].hist_p[0][i,k] for k in generators[0].Products for i in generators[0].Suppliers},sep="\t")

verbose_module.headers.print_header("Costs (c, b, h, p)")
print("\tTransport Cost")
print("\t\t",*[" "*int(np.floor((len(generators[0].V)*4-1-2)/2))+f"i{i}"+" "*int(np.floor((len(generators[0].V)*4-1-2)/2))+"   " for i in generators[-1].V],sep="\t")
for s in S:
    print(f"\t\ts{s}: ",*["-".join([" "*(3-len(str(int(generators[s-1].c[i,j]))))+str(int(generators[s-1].c[i,j])) if j != i else " 0 " for j in generators[0].V])+"\t" for i in generators[0].V], sep = "\t")

print("\n\tBackorders Cost")
print("\t\t",*[" "+f"k{k}"+" "*2 for k in generators[0].Products],sep="\t")
for s in S:
    print(f"\t\ts{s}:",*[round(generators[s-1].back_o_cost[k],2) for k in generators[0].Products],sep="\t")

print("\n\tHolding Cost")
print("\t\t",*[" "*6+f"k{k}"+" "*6  for k in generators[0].Products],sep="\t")
for s in S:
    print(f"\t\ts{s}:",*["-".join(str(generators[s-1].W_h[t][k]) for t in generators[s-1].Horizon) for k in generators[0].Products],sep="\t")

print("\n\tPurchasing Cost")
for s in S:
    print(f"\t\ts{s}:",{f"(i{i},k{k})":[generators[s-1].W_p[t][i,k] for t in generators[0].Horizon] for k in generators[0].Products for i in generators[0].Suppliers},sep="\t")

### Stochastic Parameters

In [ ]:
verbose_module.headers.print_header("Realized Demand (W_d)")
print("\t\t",*[" "*16+f"k{k}"+" "*17 for k in generators[0].Suppliers],sep="\t")
for s in S:
    print(f"\t\ts{s}:",*[[generators[s-1].W_d[t][k] for t in generators[0].Horizon] for k in generators[0].Products],sep="\t")

verbose_module.headers.print_header("Demand Sample Paths (s_paths_d)")
for t in generators[0].Horizon:
    print(f"\tDay {t}")
    for s in S:
        print(f"\t\ts{s}:",*[{f"la_t{la}":[generators[s-1].s_paths_d[t][la,ss][k] for ss in generators[0].Samples] for la in range(generators[0].sp_window_sizes[t])} for k in generators[0].Products],sep="\t")

verbose_module.headers.print_header("Realized available product (W_q)")
print("\t\t",*[" "*16+f"k{k}"+" "*17 for k in generators[0].Suppliers],sep="\t")
for s in S:
    print(f"\t\ts{s}:",*[[generators[s-1].W_q[t][i,k] for t in generators[0].Horizon] for k in generators[0].Products for i in generators[0].Suppliers],sep="\t")

verbose_module.headers.print_header("Available Quantity Sample Paths (s_paths_q)")
for t in generators[0].Horizon:
    print(f"\tDay {t}")
    for s in S:
        print(f"\t\ts{s}:",*[{f"la_t{la}":[generators[s-1].s_paths_q[t][la,ss][i,k] for ss in generators[0].Samples] for la in range(generators[0].sp_window_sizes[t])} for k in generators[0].Products for i in generators[0].Suppliers],sep="\t")


# Feasible Random Seeds Search

In [ ]:
verbose = False; agg = False; save = True; det_seed = 10

In [ ]:
#weights = [[round(i*0.1,4)]+[round((1-i*0.1)/4,4) for j in range(4)] for i in range(1,11)]
weights = [[1,0,0,0,0],[0,1,0,0,0]]; objectives = ["costs","climate","water","land","fossil"]

rewards = dict(); actions = dict(); la_decisions = dict(); states = dict(); backo = dict(); perished = dict(); norm_matrix = dict()
env_config = {'M':7,'K':7,'T':10,'F':8,'Q':1500,'S':5,'LA_horizon':4,'d_max':2000,'hist_window':60, 'theta':0.9}

seeds = [randint(1,45678987) for i in range(2000)]

attempt = 0; feas_seeds = list()

while len(feas_seeds) < 50:
    try:

        stoch_rd_seed = seeds[attempt]; det_rd_seed = 10 # Random seeds

        for w in weights:
            
            
            obj_weights = dict(zip(objectives,w))

            # Creating instance generator object
            inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params, historical_data, backorders, env_config = env_config, sustainability=True)
            inst_gen.generate_random_instance(det_rd_seed, stoch_rd_seed, q_params=q_params, p_params=p_params, d_params=d_params, h_params=h_params)

            # Resetting the environment
            states[1] = env.reset(inst_gen, return_state=True)

            done = False; day = 1
            while not done:

                ''' Objectives Pay-off Matrix and Normalization '''
                pIRPgym.Compromise_Programming.normalize_objectives(inst_gen, env, hold_cost=hold_cost)
                norm_matrix[day] = deepcopy(env.norm_matrix)

                ''' Purchase '''
                [purchase,demand_compliance], la_decisions[day] = pIRPgym.Inventory.Stochastic_Rolling_Horizon(states[day], env, inst_gen, objs=obj_weights, hold_cost=hold_cost)

                ''' Routing '''
                GA_routes,GA_obj,GA_info,GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase, inst_gen, env.t, return_top=False, rd_seed=0, time_limit=30)

                ''' Complete Action '''        
                action = {'routing':GA_routes,'purchase':purchase,'demand_compliance':demand_compliance}

                if verbose:
                    if day == 1: verbose_module.headers.print_simple_header(w)
                    print(f"\n------- Day {day}")
                    verbose_module.objectives_performance.show_normalization(inst_gen,env)
                    verbose_module.objectives_performance.show_balanced_solution(inst_gen,env,la_decisions[day],hold_cost=hold_cost)
                
                states[day+1], rewards[day], done, actions[day], (backo[day], perished[day])  = env.step(action,inst_gen,aggregated = agg)
                
                #print(f"Done day {day}")
                day += 1
            
            if save:
                other_path = True
                verbose_module.export_results.export_instance_parameters(w, 1, inst_gen, other_path)
                verbose_module.export_results.export_actions(w, 1, actions, other_path)
                verbose_module.export_results.export_lookahead_decisions(w, 1, la_decisions, other_path)
                verbose_module.export_results.export_inventory(w, 1, states, other_path)
                verbose_module.export_results.export_rewards(w, 1, rewards, other_path)
                verbose_module.export_results.export_backorders(w, 1, backo, other_path)
                verbose_module.export_results.export_perished(w, 1, perished, other_path)
                verbose_module.export_results.export_norm_matrix(w, 1, norm_matrix, other_path)
            
        feas_seeds.append(stoch_rd_seed)
        print(f"FOUND FEASIBLE SEED {len(feas_seeds)}: {stoch_rd_seed}")
        attempt += 1
                    
    except:
        
        print(f"Failed attempt {attempt} with seed {seeds[attempt]}")
        attempt += 1
        continue

file = open("C:/Users/a.rojasa55/OneDrive - Universidad de los andes/1. MIIND/Tesis/Experimentos/seeds", "wb")
dump(feas_seeds, file); file.close()

# Experiments

In [ ]:
weights = [[0.5,0.125,0.125,0.125,0.125]]; objectives = ["costs","climate","water","land","fossil"]

file = open("C:/Users/ari_r/OneDrive - Universidad de los Andes/1. MIIND/Tesis/Experimentos/seeds","rb")
seeds = load(file); file.close()

verbose = False; agg = False
for w in weights:

    for it in range(len(seeds)):

        rewards = dict()
        env_config = {'M':5,'K':7,'T':5,'F':8,'Q':1500,'S':5,'LA_horizon':4,'d_max':2000,'hist_window':60,'back_o_cost':2000}

        stoch_rd_seed, det_rd_seed = seeds[it]
        obj_weights = dict(zip(objectives,w))

        # Creating instance generator object
        inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params, historical_data, backorders, env_config = env_config, sustainability=True)
        inst_gen.generate_basic_random_instance(det_rd_seed, stoch_rd_seed, q_params=q_params, p_params=p_params, d_params=d_params, h_params=h_params)

        # Resetting the environment
        state = env.reset(inst_gen, return_state=True)

        done = False; day = 1
        while not done:

            ''' Objectives Pay-off Matrix and Normalization '''
            pIRPgym.Compromise_Programming.normalize_objectives(inst_gen, env)

            ''' Purchase '''
            [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state, env, inst_gen, objs=obj_weights)

            
            ''' Routing '''
            GA_routes,GA_obj,GA_info,GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase, inst_gen, env.t, return_top=False, rd_seed=0, time_limit=30)

            ''' Complete Action '''        
            action = {'routing':GA_routes,'purchase':purchase,'demand_compliance':demand_compliance}

            if verbose:
                if day == 1: verbose_module.headers.print_simple_header(w)
                print(f"\n------- Day {day}")
                verbose_module.objectives_performance.show_normalization(inst_gen,env)
                verbose_module.objectives_performance.show_balanced_solution(inst_gen,env,la_dec)
            
            state, rewards[day], done, real_action, _,  = env.step(action,inst_gen,aggregated = agg)
            
            day += 1

        verbose_module.export_results.export_rewards(w,it+1,rewards)
        print(f"Done {w} - {it}")